In [1]:
import torch
import sys
from swift.llm import get_model_tokenizer
dirs = ["/home/hanlv/workspace/code/research/infodemic/LLM/swift/examples/pytorch/llm",
        ".."]
for _dir in dirs:
    if _dir not in sys.path:
        sys.path.append(_dir)
import covmis
from custom import CustomModelType, CustomTemplateType
model_type = CustomModelType.mixtral_moe_7b_instruct_awq
# model_type = CustomModelType.llama_3_70b_instruct_awq
model, tokenizer = get_model_tokenizer(
    model_type, 
    load_model=False
)


2024-07-02 23:29:27,101 - modelscope - INFO - PyTorch version 2.3.0 Found.
2024-07-02 23:29:27,103 - modelscope - INFO - Loading ast index from /home/hanlv/.cache/modelscope/ast_indexer
2024-07-02 23:29:27,137 - modelscope - INFO - Loading done! Current index file version is 1.15.0, with md5 21bc0b9ccf26bcf3f4ca2e675ec8875d and a total number of 980 components indexed
[INFO:swift] Successfully registered `/home/hanlv/workspace/code/research/infodemic/LLM/swift/swift/llm/data/dataset_info.json`
[INFO:swift] Loading the model using model_dir: /home/css/models/Mixtral-8x7B-Instruct-v0.1-AWQ
[INFO:swift] model_config: MixtralConfig {
  "_name_or_path": "/home/css/models/Mixtral-8x7B-Instruct-v0.1-AWQ",
  "architectures": [
    "MixtralForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 32768,
  "model_type": "mixtral",
  

In [2]:
import jsonlines
import json
from tqdm.notebook import tqdm


sort = False
prior_knowledge_list = []

K = 5
prior_knowledge_version = "1"
model_name = "mixtral"


with open(f"train_search_llm_tmp.json", "r") as f:
    prior_knowledge_list = json.load(f)
# prior_knowledge_list = covmis.load_train_llm()


lens1 = []
lens2 = []

# for item in tqdm(prior_knowledge_list):
#     pk=item[f"prior_knowledge_{model_name}"]
#     lens1.append(len(tokenizer(pk)["input_ids"]))

for item in tqdm(prior_knowledge_list):
    # if item.get(f"prior_knowledge_{model_name}") is None:
    #     lens1.append(1000)
    #     continue

    pk=item[f"prior_knowledge_{model_name}"]
    # pk=item[f"prior_knowledge_{model_name}_v1_K=5"]

    lens1.append(len(tokenizer(pk)["input_ids"]))
    # lens2.append(len(tokenizer.tokenize(item["query"])))


  0%|          | 0/14384 [00:00<?, ?it/s]

In [3]:
cnt = 0
for i in lens1:
    if i < 240 or i > 900:
        cnt += 1
cnt


0

In [4]:
# mixtral: 240~900之外的单独跑
# llama3: 172~715
sorted(lens1, reverse=False)[:10]


[240, 240, 240, 240, 240, 240, 240, 240, 240, 240]

修改较长的句子

显卡 24GB

7B模型，长度控制在3600以下 


In [8]:
del_ids = []
for i, item in enumerate(lens1):
    if item == 151:
        del_ids.append(i)

del_ids


[5994, 8863, 11417, 13820]

In [13]:
# print(len(tokenizer(dict_list[4157]["query"])["input_ids"]))
prior_knowledge_list[14188][f"prior_knowledge_{model_name}"]


"Information 3 is relevant to the claim. According to Information 3. the claim was investigated by a Ghanaian fact-checking organization called Dubawa Ghana. They found no evidence to support the claim that Ghana's President, Nana Akufo-Addo, planned on resigning and will not contest the December 2020 elections if Ghana recorded 500 Covid-19 cases. The Director of Communications to the President, Eugene Arhin, has also denied the claim. Dubawa Ghana was unable to find any evidence of the claim being made by the Ghanaian President.\n\nThe other information provided does not directly address the claim, but they provide context to the actions of the Ghanian government and its leadership in response to the COVID-19 pandemic.\n\nTherefore, based on the information provided, the claim is FALSE, as the claim of resignation was investigated and found to be false."

In [9]:
# print(prior_knowledge_list[2278][f"prior_knowledge_{model_name}_v1_K=5"])
print(prior_knowledge_list[5994][f"prior_knowledge_{model_name}"])


Information 1 directly refutes the claim, stating that a photograph often shared in connection with food scarcity and lockdowns is actually from 2013 and unrelated to the COVID-19 pandemic. The photograph shows a baby breastfeeding on her living, mother, not a dead one. Later information provided is unrelated to the claim.

Given the evidence. FALSE

Reference(s):
Information 1: FALSE: A photograph shows a baby breastfeeding on her dead mother ... (2020-06-12)
CLAIM: A photograph shows a baby breastfeeding on her dead mother who lost her life due to food scarcity during the lockdown.


In [6]:
model_name = 'mixtral'
import json
# prior_knowledge_list = covmis.load_train_llm()
with open(f"train_search_llm_tmp.json", "r") as f:
    prior_knowledge_list = json.load(f)
def get_lst_sent(s, model_name):
    assert model_name in ["mixtral", "llama3", "solar"], "model name error"
    mixtral_pass_list = ["[Reference(s):", "Reference(s):", "Reliable sources,",
              "References:", "Sources:", "Reference List.", "Reference:", "Source:", "[References]:",
              "Reference(s, if any):", "[Reference article", "Reference links:",

              "Reasonable evidence to support the judgment:", "Reasoning behind the classification:",
              "Reasoning and evidence hierarchy:", "Reasonable evidence:", "Evidence grade:",
              "Evidence:", "Reasoning..:", "Reasoning:", "Reason:", "Rationale:", "reasons:", "Reasoning.",

              "Myevidencelock:", 
              "Factual Details:",
              "(Note:",
              "*Note:", "Note:", "Extra note.",
              "Similar symptoms:",
              "Justification:",
              "*Partial truth:",
              "Regarding the other information",
              
              "publishers:",
              "Background:",
              "More information is available",
              "Confidence:",
              "Sincerely,",
              "Please note:",
              "Action:","Explication:",
              "The NIH is beginning a new study to determine the spread of the novel",
              "Reasonable evidence to support the claim's inaccuracy includes:",
            "Comments:", "Confidence level:", "Suggested citation:", "Bibliography:",
            "The following is a summary of the relevant information regarding the claim:",
            "The following is a summary of the information provided:",
            "The references for the information used in the summary:",
            "Stealth mode:", "Caution:",
            "The following evidence was used to support the determination of FACTUALITY:",
            "The other information provided", "Reference List:", "Source list:",
            "Here are the sources used for this answer:",
            "[X](", "The reference to the other Information and articles:", "Published on 05/22/2023 02:12 PM","Thus, the claim is classified as:",
            "More specifically.", "Changes:", "Reasonable evidence to judge the correctness of the claim:", "Reference articles used in the evaluation of the claim:",
            "Meta:", "The claim wrongfully combines", "[Factualness:", "Source citations:", "The first information confirms that the", "Here’s why:",
              ]
    
    if model_name == "mixtral":
        for x in mixtral_pass_list:
            src_pos = s.lower().find(x.lower())
            if src_pos != -1:
                s = s[:src_pos].strip()

    lines = []
    for line in s.split('\n'):
        if len(line) > 0:
            lines.append(line)
    
    labels = [
        "FALSE", "TRUE",
        # "UNVERIFIED", # llama3
        ]
    for line in lines[::-1]:
        for label in labels:
            _pos = line.find(label)
            if _pos != -1:
                return line

    return lines[-1], 1
    # pos = s.rfind("\n")
    # return s[pos + 1:]

# "the CLAIM is classified as FALSE."
nn = 60
cnt = 0
for i, item in enumerate(prior_knowledge_list[:50]):
    
    a = get_lst_sent(item[f"prior_knowledge_{model_name}"], model_name)
    if isinstance(a, tuple):
        # print(i)
        # print(item[f"prior_knowledge_{model_name}_v1_K=5"])
        # print('\n\n')
        print(a[0])
        cnt += 1

cnt
    


The claim that the government of Paraíba, a state in Brazil, received 11.6 million Brazilian reais from the federal government to spend on healthcare during the COVID-19 crisis but instead, this money is being spent on artistic performances, cannot be definitively assessed as true or false with the provided information. However, the information about Brazil's history of corruption (Information 4 and 5) suggests that such misuse of public funds is a possibility.
Based on the provided information, the claim is false. The document approved by the Italian Health Ministry does not "ban" but rather discourages the routine performance of autopsies on COVID-19 patients, except for specific circumstances. The information in the article is consistent with the guidelines and protocols established by the Italian government regarding the post-mortem of COVID-19 patients.
Based on the given information, the CLAIM that the CCTV footage of a Wuhan wildlife market showing the sale and butchering of sna

15

In [434]:


lines = []
for line in prior_knowledge_list[nn][f"prior_knowledge_{model_name}_v1_K=5"].split('\n'):
    if len(line) > 0:
        lines.append(line)
lines[::-1]

['While the CLAIM does not provide specific details about the patents, it is likely that this is another conspiracy theory with no scientific basis. The cumulative evidence from the provided information suggests that the CLAIM is FALSE.',
 '3. Information 5, a Wikipedia entry from 2024-01-06, mentions a video circulating online in January 2020, which claimed to show a nurse describing a dire situation in Wuhan. However, the BBC debunked the video, stating that the woman did not claim to be a nurse or doctor, and her attire did not match medical staff in Hubei. This suggests that similar videos with false claims were circulating online during the pandemic.',
 '2. Information 3, an article from 2020-05-12, discusses a viral video with harmful claims, which was removed by platforms, but still managed to reach over 8 million views. This could be the same video referred to in the CLAIM.',
 '1. Information 1, a fact-check from 2020-04-10, states that scientific evidence indicates the SARS-Co

In [ ]:

ok_list = [
    "information",
    "* TRUE for the \"COVID-19 Daily".lower(),
    "claim", "impact", "aily",
    "classified as", "clarifications",
    "summary","conclusion","summarized",
    "is", "ydroxychloroquine", 
    "formation 1","formation 2","formation 3","formation 4","formation 5", "5",
    "discussion", "decision", "assessment", "classification", "covid-19", "covid--19", "clarification",
    "content", "force", "includes", "title", "'cabin'", "check",
    "rating", "social", "infodemic", "SARS-CoV-2".lower(),
    "uestions", "as follows", "analysis", "idsa", "niversities","include", "auci",
    "https", "date", "verification", "verdict", "contact", "symptoms", "says", "knowledge",
    "correctness", "DETERMINATION".lower(), "assertion", "system'", "system", "rally",
    "are", "RULING".lower(), "as", "published", "coronavirus", "false", "COMMENTARY".lower(),
    "explanation", "pandemic", "igerians", "you", "publication", "pets", "hype","this'","laim",
    "topic", "covid", "14", ")", "act-check findings", "eracity", "vidence.1", "response",
    "eaths", "diarrhea", "inaccuracy", "caveat", "ockdown", "classifications", "STATUS".lower(),
    "that", "ates", "ays", "judgment", "true", "be", "doi", "biotics", "articles", "mostly",
    "trump", "details", "answer", "theories", "following", "2020-21", "outcomes", "tudy",
    "ives", "judgement", "ntroduction", "ine", "CLUII".lower(), "ormula",
    "johnshopkins", "diagnostic remarks", "http", "exercises", "recommendation", "leak",
    "coronaviruses", "summarize", "istancing", "utlook", "orrection", "remains","ongolia",
    "abric", "43", "28", "reasonings", "esult", "landemic", "DRUGS".lower(), "esting",
    "ositive", "therefore", "used", "06", "incidents", "onuses", "above", "masks", "opposite",
    "involves", "ictims", "informations", "treatments", "insights", "angerous", "pread",
    "concepts", "ight", "valuation", "provided", "said", "odels", "partments", "fizer", "examination",
    "context", "tests", "oll", "keypoint", "lasification", "diseases", "ighlighted", "iktionary",
    "including", "**CLAIM".lower(), "ovations", "contents", "actual", "co-ki”", "recipients",
    "remarks", "84", "debunked", "accuracy", "elements", "claire", "china", "fact", "display", "explain",
    "turkish", "tv", "ones", "mailto", "opinion", "validity", "a 2", "aev", "well", "ystery", "color",
    "true*", "hmadinejad", "ay 2", "lunchtime", "~", "processes**", "xpert", "sleep", "considering",
    "misleading", "factors", "cdc", "lait", "taught", "findings", "ruth", "rick", "oax", "20",

]
for i, item in enumerate(prior_knowledge_list):
    
    s = item[f"prior_knowledge_{model_name}"]
    nxt = False
    for x in pass_list:
        src_pos = s.lower().find(x.lower())
        if src_pos != -1:
            nxt = True
            break

    if nxt:
        continue
    pos1 = s.rfind(":")
    if pos1 == -1:
        continue
    pos2 = s[:pos1].rfind("\n")
    for s_ok in ok_list:
        if s[pos2+1:pos1].lower().endswith(s_ok) or s[pos2+1:pos1].lower().startswith(s_ok):
            nxt = True
            break
    if nxt:
        continue
    print(i)
    # print(s[pos2+1:pos1])
    print(s[pos2+1:])
    print('#'*50)
    print(s)
    break
